In [16]:
from pathlib import Path

data_path = Path("data/")
image_path = data_path / "events"
train_dir = image_path / "train_scaled"
test_dir = image_path / "test_scaled"

train_dir, test_dir
output_dir = 'output'

In [17]:

import torch
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
if torch.cuda.is_available(): 
    dev = "cuda:0" 
else: 
    dev = "cpu" 
device = torch.device(dev)

In [19]:
import matplotlib.pyplot as plt
import torch

from torch import nn
from torchvision import transforms

from torchvision.models import EfficientNet_V2_S_Weights, efficientnet_v2_s
from torchvision.models import ResNet152_Weights, resnet152

weights = EfficientNet_V2_S_Weights.DEFAULT
model = efficientnet_v2_s(weights=weights).to(device)
# weights = ResNet152_Weights.DEFAULT
# model = resnet152(weights=weights).to(device)



In [20]:
auto_transforms = weights.transforms()
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [21]:
from torchvision import datasets
train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=auto_transforms, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)
test_data = datasets.ImageFolder(root=test_dir, 
                                 transform=auto_transforms)
# train_data = datasets.ImageFolder(root=train_dir, # target folder of images
#                                   transform=train_transform, # transforms to perform on data (images)
#                                   target_transform=None) # transforms to perform on labels (if necessary)
# test_data = datasets.ImageFolder(root=test_dir, 
#                                  transform=val_transform)


In [22]:

class_names = train_data.classes
class_names

['Combat',
 'DestroyedBuildings',
 'Fire',
 'Humanitarian Aid and rehabilitation',
 'Military vehicles and weapons']

In [23]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=32,shuffle=False)

In [24]:
# # Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
for param in model.features.parameters():
    param.requires_grad = False

# for param in model.parameters():
#     param.requires_grad = False

In [25]:
# Get the length of class_names (one output unit for each class)
output_shape = len(class_names)
# torch.autograd.set_detect_anomaly(True)
# Recreate the classifier layer and seed it to the target device

model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=output_shape, bias=True),
    # nn.ReLU6(),   
).to(device)

In [26]:
from torch import nn
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1) # this is also called "criterion"/"cost function" in some places
# optimizer = torch.optim.RMSprop(params=model.parameters(), lr=0.01)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

In [27]:
torch.cuda.set_device(0)

In [28]:
next(model.parameters()).is_cuda

True

In [29]:
# Set the random seeds
try:
    from going_modular.going_modular import engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !move pytorch-deep-learning\going_modular .
    !rmdir /s /q pytorch-deep-learning
    from going_modular.going_modular import engine

# Start the timer
from timeit import default_timer as timer 
start_time = timer()

# Setup training and save the results
results = engine.train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=25,
                       device=device,
                       save_path='best.tf')

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")


  0%|          | 0/25 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.5627 | train_acc: 0.3375 | test_loss: 1.3873 | test_acc: 0.7500
Epoch: 2 | train_loss: 1.2225 | train_acc: 0.7812 | test_loss: 1.0161 | test_acc: 0.9375
Epoch: 3 | train_loss: 0.9217 | train_acc: 0.8938 | test_loss: 0.8161 | test_acc: 0.9167
Epoch: 4 | train_loss: 0.7600 | train_acc: 0.9125 | test_loss: 0.7189 | test_acc: 0.9167
Epoch: 5 | train_loss: 0.6757 | train_acc: 0.9375 | test_loss: 0.6777 | test_acc: 0.9167
Epoch: 6 | train_loss: 0.6601 | train_acc: 0.8969 | test_loss: 0.6443 | test_acc: 0.9375
Epoch: 7 | train_loss: 0.6152 | train_acc: 0.9500 | test_loss: 0.6300 | test_acc: 0.9375
Epoch: 8 | train_loss: 0.5920 | train_acc: 0.9344 | test_loss: 0.6197 | test_acc: 0.9479
Epoch: 9 | train_loss: 0.5879 | train_acc: 0.9531 | test_loss: 0.6110 | test_acc: 0.9271
Epoch: 10 | train_loss: 0.5781 | train_acc: 0.9500 | test_loss: 0.5984 | test_acc: 0.9375
Epoch: 11 | train_loss: 0.5640 | train_acc: 0.9437 | test_loss: 0.5969 | test_acc: 0.9375
Epoch: 12 | train_l

In [30]:
torch.save(model, 'model.tf')
torch.save(model.state_dict(), 'weights.tf')